# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
N. Wang  ->  N. Wang  |  ['N. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
P. Griveaud  ->  P. Griveaud  |  ['P. Griveaud']
J. Liu  ->  J. Liu  |  ['J. Liu']
D. Christie  ->  D. Christie  |  ['D. Christie']
J. Li  ->  J. Li  |  ['J. Li']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']


F. Walter  ->  F. Walter  |  ['F. Walter']
Arxiv has 90 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.22737


extracting tarball to tmp_2506.22737...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.22765
extracting tarball to tmp_2506.22765... done.
Retrieving document from  https://arxiv.org/e-print/2506.22904


extracting tarball to tmp_2506.22904... done.
Retrieving document from  https://arxiv.org/e-print/2506.23331
extracting tarball to tmp_2506.23331...

...

Found 90 bibliographic references in tmp_2506.23891/mnras_K2-18b_PartI.bbl.
Retrieving document from  https://arxiv.org/e-print/2506.23915


extracting tarball to tmp_2506.23915...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.23931
extracting tarball to tmp_2506.23931... done.
Retrieving document from  https://arxiv.org/e-print/2506.24128


extracting tarball to tmp_2506.24128... done.


Found 110 bibliographic references in tmp_2506.24128/faintqso.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.23891-b31b1b.svg)](https://arxiv.org/abs/2506.23891) | **Three-dimensional Transport-induced Chemistry on Temperate sub-Neptune K2-18b, Part I: the Effects of Atmospheric Dynamics**  |
|| <mark>J. Liu</mark>, <mark>D. Christie</mark>, J. Yang |
|*Appeared on*| *2025-07-01*|
|*Comments*| *19 pages, 17 figures, accepted to MNRAS*|
|**Abstract**|            The low equilibrium temperatures of temperate sub-Neptunes lead to extremely long chemical timescales in their upper atmospheres, causing the abundances of chemical species to be strongly shaped by atmospheric transport. Here, we used a three-dimensional (3D) general circulation model involving a passive tracer to investigate the atmospheric circulation and 3D transport of temperate gas-rich sub-Neptunes, using K2-18b as an example. We model K2-18b as a synchronous or asynchronous rotator, exploring spin-orbit resonances (SOR) of 2:1, 6:1, and 10:1. We find that the strong absorption of CO$_2$ and CH$_4$ induces a detached convective zone between 1 and 5 bar, resulting in strong vertical mixing at these levels. The upper atmosphere is dominated by eastward winds (an equatorial superrotating jet present in all simulations), leading to warmer evening terminators and approximately 20% higher passive tracer mass mixing ratios compared to the morning terminators. Rotation rates have minimal impact on the strength of global mean vertical mixing, but significantly influence the latitudinal distribution of passive tracers. For synchronous, 2:1 SOR, and 6:1 SOR simulations, passive tracers are more abundant in the upwelling branches at latitudes within 60$^\circ$, while for the 10:1 SOR simulation, strong transient eddies at high latitudes (>70$^\circ$) between 0.1 to 1 bar can transport passive tracers upward from the deep atmosphere, making them more abundant there, despite their alignment with the downwelling branch of the large-scale circulation. This study focuses on the atmospheric dynamics and its influence on passive tracer transport, while a follow-up paper will incorporate active chemical species.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.24128-b31b1b.svg)](https://arxiv.org/abs/2506.24128) | **Beneath the Surface: >85% of z>5.9 QSOs in Massive Host Galaxies are UV-Faint**  |
|| R. Bouwens, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-07-01*|
|*Comments*| *13 pages, 6 figures, 4 tables, submitted to A&A*|
|**Abstract**|            We use [CII] observations of a large QSO sample to segregate sources by host galaxy mass, aiming to identify those in the most massive hosts. [CII] luminosity, a known tracer of molecular gas, is taken as a proxy for host mass and used to rank 190 QSOs at z>5.9, spanning a 6-mag UV luminosity range (-22<Muv<-28). Particularly valuable are ALMA data from a cycle-10 CISTERN program, providing [CII] coverage for 46 UV-faint (M_{UV,AB}>-24.5) and 25 especially UV-faint (Muv>-23.5) QSOs, improving statistics by 5x and 6x, respectively. Taking massive host galaxies to be those where L[CII]>1.8x10^9 Lsol (median L[CII] of UV-bright QSOs), we identify 61 QSOs, including 13 which are UV-faint and 7 especially UV-faint. Using these selections and recent QSO luminosity functions (LFs), we present the first characterization of UV luminosity distribution for QSOs in massive host galaxies and quantify [CII] LFs for both UV-bright and UV-faint QSOs. While ~3% of massive-host QSOs are UV-bright (Muv<-26), >~85% are UV-faint (Muv>-24.5). This wide dispersion in UV luminosities reflects variations in dust obscuration, accretion efficiency, and black hole mass. Though spectroscopy is needed for definitive conclusions, black hole mass appears to be the dominant factor driving variations in the UV luminosity, based on 34 [CII]-luminous (L[CII]>1.8x10^9 Lsol) QSOs distributed across a ~3-mag baseline in UV luminosity and with measured MBH. At Muv~-23, the median extrapolated log10 (MBH/Msol) is 8.1+/-0.4, consistent with the local relation. SMBHs in UV-bright QSOs thus appear to be ~15(-9)(+25)x more massive than typical for massive host galaxies at z~6.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.22737-b31b1b.svg)](https://arxiv.org/abs/2506.22737) | **Exploring the signature of assembly bias and modified gravity using small-scale clusterings of galaxies**  |
|| <mark>Y. Wang</mark>, Z. Zhai, X. Yang, J. L. Tinker |
|*Appeared on*| *2025-07-01*|
|*Comments*| *16 pages, 8 figures*|
|**Abstract**|            We apply a halo velocity bias model, $\gamma_{f}$, within the Aemulus simulation suite for General Relativity (GR) to investigate its efficacy in identifying the signature of assembly bias and Modified Gravity (MG). In the investigation of assembly bias, utilizing galaxy clustering data ranging from scales of $0.1 - 60 \text{Mpc}\,h^{-1}$, we discover that our emulator model accurately recreates the cosmological parameters, $\Omega_m$ and $\sigma_8$, along with the velocity bias $\gamma_{f}$, staying well within the 1-$\sigma$ error margins, provided that assembly bias is considered. Ignoring assembly bias can considerably alter our model constraints on parameters $\Omega_m$ and $\sigma_8$ if the test sample includes assembly bias. Using our emulator for MG simulations, which encompasses two Dvali-Gabadadze-Porrati models (DGP; N1, N5) and two $f(R)$ models (F4, F6), we can effectively identify a robust signature of modified gravity, for models such as DGP-N1 and $f(R)$-F4, as indicated by a noticeable deviation of $\gamma_{f}$ from unity. Using the velocity dispersion of dark matter halos to effectively represent the underlying strength of the velocity field of these MG simulations, we find that the simple $\gamma_{f}$ model can recover the truth with minimal bias. These evaluations indicate that our simple halo-velocity bias model is capable of detecting significant MG characteristics, although additional methodologies should be pursued to improve model constraints.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.22765-b31b1b.svg)](https://arxiv.org/abs/2506.22765) | **Timing results of 22 years for PSR J0922+0638**  |
|| P. Liu, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-07-01*|
|*Comments*| *14 pages, 8 figures, 4 tables, ApJ (2025) accepted*|
|**Abstract**|            We conducted a timing analysis of PSR J0922+0638 (B0919+06) using data from the Nanshan 26 m radio telescope and the MeerKAT telescope, spanning from January 2001 to March 2023. During this 22-year period, we discovered a previously unreported small glitch (glitch 1) before the well-known large glitch (glitch 2), occurring at ${\rm MJD} \sim 53325(3)$, with a frequency jump amplitude of $\Delta \nu/\nu \sim 0.79(6) \times 10^{-9}$. We also identified ten slow glitch events, half of which were newly detected. These slow glitches occurred quasi-periodically, with an average interval of approximately 553(21) days, fractional frequency changes ranging from $\Delta \nu/\nu \sim 1.13(1) \times 10^{-9}$ to $4.08(5) \times 10^{-9}$, and a maximum fractional change in the first derivative of the frequency of $\Delta \dot{\nu}/\dot{\nu} \sim -4.6 \times 10^{-3}$. Additionally, our timing noise analysis reveals a change in the spectral index for noise power before and after glitch 2, with values of $-6.0$ and $-5.3$, respectively, likely due to this large glitch. Throughout the entire observation period, the first derivative of the spin frequency ($\dot{\nu}$) showed a periodic structure. The possible modulation period was estimated to be 537(24) days before the 700-day data gap at MJD 56716 and 600(58) days afterward. We discuss the periodic oscillations in pulsar rotation as a possible manifestation of spin-down noise and quasi-periodic slow glitches.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.22904-b31b1b.svg)](https://arxiv.org/abs/2506.22904) | **The effectively optically thin accretion flow and its implication in supermassive black holes**  |
|| M. Liu, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-01*|
|*Comments*| *11 pages, 9 figures, accepted for publication in MNRAS*|
|**Abstract**|            Based on a unified description of various accretion flows, we find a long-ignored solution - the effectively optically thin accretion flow, occurring at accretion rates around Eddington value. As a consequence of radiation-pressure dominance, the density in a standard thin disc (SSD) decreases with the increase of accretion rates, making the innermost region effectively optically thin. Further increase in accretion rate leads to a rise of the temperature so that the Compton cooling is able to balance the accretion released energy. We demonstrate that the effectively optically thin flow is characterized by moderate temperature and large scattering optical depth, producing a multi-color Wien spectrum. For an appropriate accretion rate, the accretion flow transforms from an outer SSD to an inner effectively optically thin flow. Thus, the spectra of the whole accretion flow exhibit two components, i.e., a multi-color Wien spectrum at higher frequency and a multi-color blackbody, the former could provide an alternative origin of soft X-ray excess or formation of warm corona in active galactic nuclei (AGNs). Our stability analysis proves it is thermally stable and viscously unstable, indicating its existence in accreting systems. We show that effectively optically thin accretion flow exists in supermassive black holes for accretion rates around 0.1 to 10 times Eddington value, bridging the SSD at low accretion rates and slim disc at high rates. By comparing the predictions and average spectra of AGN, we constrain the viscosity parameter to be $\alpha \sim 0.03$, in good agreement with that derived from observed variability.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.23915-b31b1b.svg)](https://arxiv.org/abs/2506.23915) | **Exploring the Wolf Approach to Constraining NIR Extinction Laws in the Corona Australis Molecular Cloud**  |
|| B. Jiang, <mark>J. Li</mark>, X. Chen |
|*Appeared on*| *2025-07-01*|
|*Comments*| *Accepted for publication in AJ*|
|**Abstract**|            The viability of the star count (Wolf) method is assessed as a means of constraining the near-infrared (NIR) extinction law toward the Corona Australis molecular cloud. Using deep $JHK_S$ photometry from the VISIONS survey, extinction maps with 1 arcmin spatial resolution are constructed. The derived extinction ratios are $A_J/A_H=1.73\pm0.07$, $A_H/A_{K_S}=1.70\pm0.11$, and $A_J/A_{K_S}=3.02\pm0.22$, which are consistent with Galactic literature means. Assuming a power-law form ($A_\lambda \propto \lambda^{-\alpha}$) for the NIR extinction law, we derive indices of $\alpha\approx 2.0$ across all wavelength combinations, with no statistically significant wavelength dependence throughout the NIR wavelength range. While spatial variations in extinction properties are tentatively observed across the cloud, concerns persist regarding the impact of photometric completeness, and the role of reference field selection. Continued research is required to refine the approach, and scrutinize the veracity of potential extinction law variations over a more expansive region of sky.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.23931-b31b1b.svg)](https://arxiv.org/abs/2506.23931) | **Sagittarius A* near-infrared flares polarization as a probe of space-time I: Non-rotating exotic compact objects**  |
|| N. Aimar, J. L. Rosa, H. L. Tamm, <mark>P. Garcia</mark> |
|*Appeared on*| *2025-07-01*|
|*Comments*| *14 pages, 9 figures, 7 tables; comments welcome*|
|**Abstract**|            The center of our galaxy hosts Sagittarius~A*, a supermassive compact object of $\sim 4.3\times 10^6$ solar masses, usually associated with a black hole. Nevertheless, black holes possess a central singularity, considered unphysical, and an event horizon, which leads to loss of unitarity in a quantum description of the system. To address these theoretical inconsistencies, alternative models, collectively known as exotic compact objects, have been proposed. In this paper, we investigate the potential detectability of signatures associated with non-rotating exotic compact objects within the Sgr~A* polarized flares dataset, as observed through GRAVITY and future instruments. We examine a total of eight distinct metrics, originating from four different categories of static and spherically symmetric compact objects: Black Holes, Boson stars, Fluid spheres, and Gravastars. Our approach involves utilizing a toy model that orbits the compact object in the equatorial plane, at the Schwarzschild-Keplerian velocity. Using simulated astrometric and polarimetric data with present GRAVITY and future GRAVITY+ uncertainties, we fit the datasets across all metrics examined. We evaluated the detectability of the metric for each dataset based on the resulting $\chi^2_\mathrm{red}$ and BIC-based Bayes factors. Plunge-through images of ECOs affect polarization and astrometry. With GRAVITY's present uncertainties, only a compact boson-star model is discernible. GRAVITY+'s improved sensitivity allows detection of most exotic compact object models. However, enhancing the astrophysical complexity of the hot spot model diminishes these outcomes. Presently, GRAVITY's uncertainties limit us to detecting just one exotic compact object metric. With GRAVITY+'s enhanced sensitivity, we can expect to uncover additional exotic compact object models and use Sgr~A* as a laboratory for fundamental physics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.23331-b31b1b.svg)](https://arxiv.org/abs/2506.23331) | **On inertial forces (indirect terms) in problems with a central body**  |
|| A. Crida, et al. -- incl., <mark>P. Griveaud</mark> |
|*Appeared on*| *2025-07-01*|
|*Comments*| *12 pages, accepted for publication in the Open Journal of Astrophysics*|
|**Abstract**|            Gravitational systems in astrophysics often comprise a body -- the primary -- that far outweights the others, and which is taken as the centre of the reference frame. A fictitious acceleration, also known as the indirect term, must therefore be added to all other bodies in the system to compensate for the absence of motion of the primary. In this paper, we first stress that there is not one indirect term but as many indirect terms as there are bodies in the system that exert a gravitational pull on the primary. For instance, in the case of a protoplanetary disc with two planets, there are three indirect terms: one arising from the whole disc, and one per planet. We also highlight that the direct and indirect gravitational accelerations should be treated in a balanced way: the indirect term from one body should be applied to the other bodies in the system that feel its direct gravitational acceleration, and only to them. We point to situations where one of those terms is usually neglected however, which may lead to spurious results. These ideas are developed here for star-disc-planets interactions, for which we propose a recipe for the force to be applied onto a migrating planet, but they can easily be generalized to other astrophysical systems.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error nothing to repeat at position 15</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.23891/./Figures/rhomomentum_conv_10to1_100days_r.png', 'tmp_2506.23891/./Figures/rhomomentum_conv_tidally-locked_100days_r.png', 'tmp_2506.23891/./Figures/wind_structure_uvw_heliocenteric_v.png']
copying  tmp_2506.23891/./Figures/rhomomentum_conv_10to1_100days_r.png to _build/html/
copying  tmp_2506.23891/./Figures/rhomomentum_conv_tidally-locked_100days_r.png to _build/html/
copying  tmp_2506.23891/./Figures/wind_structure_uvw_heliocenteric_v.png to _build/html/
exported in  _build/html/2506.23891.md
    + _build/html/tmp_2506.23891/./Figures/rhomomentum_conv_10to1_100days_r.png
    + _build/html/tmp_2506.23891/./Figures/rhomomentum_conv_tidally-locked_100days_r.png
    + _build/html/tmp_2506.23891/./Figures/wind_structure_uvw_heliocenteric_v.png
found figures ['tmp_2506.24128/./extrap-mbh.png', 'tmp_2506.24128/./extrap-erdf.png', 'tmp_2506.24128/./cii_muv.png']
copying  tmp_2506.24128/./extrap-mbh.png to _build/html/
copying  tmp_2506.24128/./extrap-erdf.png 

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\revise}[1]{\textcolor{red}{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Three-dimensional Transport-induced Chemistry on Temperate sub-Neptune K2-18b, Part I: the Effects of Atmospheric Dynamics

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.23891-b31b1b.svg)](https://arxiv.org/abs/2506.23891)<mark>Appeared on: 2025-07-01</mark> -  _19 pages, 17 figures, accepted to MNRAS_

</div>
<div id="authors">

<mark>J. Liu</mark>, <mark>D. Christie</mark>, J. Yang

</div>
<div id="abstract">

**Abstract:** The low equilibrium temperatures of temperate sub-Neptunes lead to extremely long chemical timescales in their upper atmospheres, causing the abundances of chemical species to be strongly shaped by atmospheric transport. Here, we used a three-dimensional (3D) general circulation model involving a passive tracer to investigate the atmospheric circulation and 3D transport of temperate gas-rich sub-Neptunes, using K2-18b as an example. We model K2-18b as a synchronous or asynchronous rotator, exploring spin-orbit resonances (SOR) of 2:1, 6:1, and 10:1. We find that the strong absorption of $CO_2$ and $CH_4$ induces a detached convective zone between 1 and 5 bar, resulting in strong vertical mixing at these levels. The upper atmosphere is dominated by eastward winds (an equatorial superrotating jet present in all simulations), leading to warmer evening terminators and approximately 20 $\%$ higher passive tracer mass mixing ratios compared to the morning terminators. Rotation rates have minimal impact on the strength of global mean vertical mixing, but significantly influence the latitudinal distribution of passive tracers. For synchronous, 2:1 SOR, and 6:1 SOR simulations, passive tracers are more abundant in the upwelling branches at latitudes within 60 $^\circ$ , while for the 10:1 SOR simulation, strong transient eddies at high latitudes ( $>$ 70 $^\circ$ ) between 0.1 to 1 bar can transport passive tracers upward from the deep atmosphere, making them more abundant there, despite their alignment with the downwelling branch of the large-scale circulation. This study focuses on the atmospheric dynamics and its influence on passive tracer transport, while a follow-up paper will incorporate active chemical species.

</div>

<div id="div_fig1">

<img src="tmp_2506.23891/./Figures/rhomomentum_conv_10to1_100days_r.png" alt="Fig7" width="100%"/>

**Figure 7. -** Mechanisms of the formation of the equatorial superrotating jet and high-latitude jets for the 10:1 SOR simulation. (a) Zonal-mean zonal wind at 100 days. (b) Zonal-mean zonal wind accelerations at $10^{-4}$ bar due to horizontal (solid) and vertical (dashed) transient eddies (blue), stationary eddies (orange), and mean flow (purple) in equation (\ref{momentum equation new}). The black line in panel (a) represents the sum of these six terms. (c)-(h): Contributions to zonal wind acceleration from horizontal transient eddies, horizontal stationary eddies, horizontal mean flow, vertical transient eddies, vertical stationary eddies, and vertical mean flow in equation (\ref{momentum equation new}). Contour lines in panels (e) and (h) depict the zonal-mean mass streamfunction, with arrows indicating the direction of mean circulation and angular momentum transport by the mean flow. To enhance visibility of the mass streamfunction in the upper atmosphere, two contour intervals are applied: for pressures smaller than 0.006 bar, the interval is $8 \times 10^8$ kg s$^{-1}$ with limits of $-2 \times 10^9$ and $2 \times 10^9$ kg s$^{-1}$, while for pressures larger than 0.006 bar, the interval is $4 \times 10^9$ kg s$^{-1}$ with limits of $-2 \times 10^{10}$ and $2 \times 10^{10}$ kg s$^{-1}$. Results in panels (b)-(h) are derived from simulated data over the first 100 days, with an output interval of 10 days. (*fig:momentum_ini_10to1*)

</div>
<div id="div_fig2">

<img src="tmp_2506.23891/./Figures/rhomomentum_conv_tidally-locked_100days_r.png" alt="Fig6" width="100%"/>

**Figure 6. -** Mechanism for the formation of the equatorial superrotating jet for the synchronous simulation. (a) The zonal-mean zonal wind at 100 days. (b) Zonal-mean zonal wind accelerations at 0.001 bar due to horizontal (solid) and vertical (dashed) transient eddy (blue), stationary eddy (orange), and mean flow (purple) in equation (\ref{momentum equation new}). The black line in panel (b) is the sum of these six terms. (c) Horizontal distribution of the stationary eddy velocity at 0.001 bar, where $\overline{u}^*$ is depicted as the coloured contour and $\overline{\rho v}^*/[\overline{\rho}]$ is shown as the contour lines, with an interval of 40 m s$^{-2}$ and limits of -200 and 200 m s$^{-2}$. (d)-(i): Contributions to zonal wind acceleration from horizontal transient eddies, horizontal stationary eddies, horizontal mean flow, vertical transient eddies, vertical stationary eddies, and vertical mean flow in equation (\ref{momentum equation new}). Contour lines in panels (f) and (i) show the zonal-mean mass streamfunction, with arrows indicating the direction of mean circulation and angular momentum transport by the mean flow. The contour lines have an interval of 10$^{10}$ kg s$^{-1}$, with limits of -5$\times$10$^{10}$ and 5$\times$10$^{10}$ kg s$^{-1}$. Results in panels (b)-(i) are derived from simulated data over the first 100 days, with an output interval of 10 days. (*fig:momentum_ini_t*)

</div>
<div id="div_fig3">

<img src="tmp_2506.23891/./Figures/wind_structure_uvw_heliocenteric_v.png" alt="Fig5" width="100%"/>

**Figure 5. -** Horizontal distribution of zonal wind ((a)--(d)), meridional wind ((e)--(h)), and vertical wind ((i)--(l)) at 0.001 bar. The streamlines indicate the direction of the horizontal flow. The results are transforming in the heliocentric frame to show the wind patterns more clearly. The black star-shaped markers indicate the location of the substellar point. Rows from the top to the bottom are results from simulations of synchronous, 2:1 SOR, 6:1 SOR, and 10:1 SOR. (*fig:uvw*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.23891"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Beneath the Surface: $>$85\% of z$>$5.9 QSOs in Massive Host Galaxies are $UV$-Faint

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.24128-b31b1b.svg)](https://arxiv.org/abs/2506.24128)<mark>Appeared on: 2025-07-01</mark> -  _13 pages, 6 figures, 4 tables, submitted to A&A_

</div>
<div id="authors">

R. Bouwens, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We use [ CII ] ${158\mu\text{m}}$ observations of a large QSO sample to  segregate sources by host galaxy mass, aiming to identify those in  the most massive hosts. [ CII ] luminosity, a known tracer of  molecular gas, is taken as a proxy for host mass and used to rank  190 QSOs at $z>5.9$ , spanning a 6-mag UV luminosity range  ( $-22<M{UV,AB}<-28$ ). Particularly valuable are ALMA data from a  cycle-10 CISTERN program, providing [ CII ] coverage for 46 $UV$ -faint  ( $M_{UV,AB}>-24.5$ ) and 25 especially $UV$ -faint ( $M_{UV,AB}>-23.5$ )  QSOs, improving statistics by 5 $\times$ and 6 $\times$ , respectively.  Taking massive host galaxies to be those where $L_{\text{[CII]}}$ $>1.8\times10^9$ L $_\odot$ (median $L_{[CII]}$ of $UV$ -bright  QSOs), we identify 61 QSOs, including 13 which are $UV$ -faint and 7  especially $UV$ -faint.  Using these selections and recent QSO  luminosity functions (LFs), we present the first characterization of $UV$ luminosity distribution for QSOs in massive host galaxies and  quantify [ CII ] LFs for both $UV$ -bright and $UV$ -faint QSOs.  While $\sim$ 3 \% of massive-host QSOs are $UV$ -bright ( $M_{UV,AB}<-26$ ), $\gtrsim$ 85 \% are $UV$ -faint ( $M_{UV,AB}>-24.5$ ).  This wide  dispersion in $UV$ luminosities reflects variations in dust  obscuration, accretion efficiency, and black hole mass. Though  spectroscopy is needed for definitive conclusions, black hole mass  appears to be the dominant factor driving variations in the $UV$ luminosity, based on 34 [ CII ] -luminous  ( $L_{\text{[CII]}}>1.8\times10^9$ L $_{\odot}$ ) QSOs distributed  across a $\sim$ 3-mag baseline in $UV$ luminosity and with measured $M_{BH}$ .  At $M_{UV,AB}\sim-23$ , the median extrapolated $\log_{10}  (M_{BH}/M_{\odot})$ is $8.1\pm0.4$ , consistent with the local  relation.  SMBHs in $UV$ -bright QSOs thus appear to be $\sim$ 15 $_{-9}^{+25}\times$ more massive than typical for massive  host galaxies at z $\sim$ 6.

</div>

<div id="div_fig1">

<img src="tmp_2506.24128/./extrap-mbh.png" alt="Fig5" width="100%"/>

**Figure 5. -** _(left)_ Median measured $M_{BH}$(\textit{solid black
    circles}) from the literature for QSOs in Massive Host Galaxies
  with $L_{\text{[CII]}}>$1.8$\times$10$^9$ L$_{\odot}$ vs. $UV$
  luminosity.  The smaller solid grey points show measured $M_{BH}$
  vs. $M_{UV}$ for the massive host galaxy sample.  The $M_{BH}$
  measurements for a $M_{UV,AB}\sim-24.1$ QSO at $z=7.07$ ([Izumi, Matsuoka and Fujimoto 2021]())  is shown with the grey star.  The dotted diagonal lines indicate $M_{BH}$'s corresponding to Eddington ratios $\lambda_{Edd}$ of 1, 0.1, and 0.01.  There is a strongly
  suggestive correlation between the black hole masses of QSOs in
  massive host galaxies and their $UV$ luminosities, suggesting that
  $UV$ brightness of QSOs in massive host galaxies is largely driven by
  the BH mass.  The open red circle shows the extrapolated
  $M_{BH}$ at $M_{UV,AB}\sim-23.0$ mag, together with the $1\sigma$
  uncertainty based on the extrapolation.  _(right)_
  Distribution of $M_{BH}$ in massive host galaxies at $z>6$ in
  $UV$-bright ($M_{UV,AB}<-26$) QSOs (_black line_) and including
  QSOs down to a $UV$ luminosity of $-$22.0 mag (_red line_).
  The precise mass where the $M_{BH}$ distribution peaks has a 1$\sigma$
  uncertainty of $\pm$0.4 dex (_red arrows_) and is sensitive to the $M_{UV,AB}=-22$ cut-off the QSO LF we assume and also the best-fit slope of the $M_{BH}$ vs. $M_{UV}$ relation.  The purple bracketed region indicates the approximate
  range of $M_{BH}$ for QSOs in similar mass host galaxies in the
  local universe (e.g.,  ([ and Kormendy 2013]()) : KH13).  The median $M_{BH}$ for
  QSOs in massive host galaxies at $z\sim6$ appears to be consistent
  with the local value (see \S\ref{sec:mbhdist}), but the uncertainties
  are too large to be sure.  Further measurements
  of $M_{BH}$ for the $UV$-faint QSOs in massive host galaxies at
  $z\sim6$ are required to resolve this open
  question. (*fig:mbhdist*)

</div>
<div id="div_fig2">

<img src="tmp_2506.24128/./extrap-erdf.png" alt="Fig6" width="100%"/>

**Figure 6. -** _(left)_ Median inferred $\lambda_{Edd}$(_solid black circles_) from the literature for QSOs in
  Massive Host Galaxies with $L_{\text{[CII]}}>$1.8$\times$10$^9$
  L$_{\odot}$ vs. $UV$ luminosity.  The grey points and grey star are
  as in Figure \ref{fig:mbhdist}.  No strong correlation is seen
  between the $UV$ luminosity of QSOs in massive host galaxies and
  their accretion efficiency $\lambda_{Edd}$.  As such, it would
  appear that $M_{BH}$ is the dominant factor impacting the $UV$
  brightness (or $L_{bol}$) of $z\sim6$-7 QSOs and the duty cycle
  cycle (i.e., variations in $\lambda_{Edd}$ has a more limited
  impact.  The open red circle shows the extrapolated $M_{BH}$ at $-$23.0 mag, together with the $1\sigma$ uncertainty
  based on the extrapolation.  _(right)_ Distribution of
  $\lambda_{Edd}$ in massive host galaxies at $z>6$ in $UV$-bright
  ($M_{UV,AB}<-26$) QSOs (_black line_) and including QSOs down
  to a $UV$ luminosity of $-$22.0 mag (_red line_).  Given
  the large uncertainties in the precise dependence $\lambda_{Edd}$
  shows on $UV$ luminosity, the peak of the distribution is very
  uncertain ($\pm$0.4 dex: _red arrows indicate the 1$\sigma$ uncertainties_). (*fig:erdist*)

</div>
<div id="div_fig3">

<img src="tmp_2506.24128/./cii_muv.png" alt="Fig1" width="100%"/>

**Figure 1. -** (_upper panel_) Measured [CII] luminosities
     vs. $M_{UV}$ luminosities for $z>5.9$ QSOs from CISTERN
     (_red circles_) and from the literature (_black circles_).  The solid downward pointing triangles indicate
     $5\sigma$ upper limits on the [CII] luminosities of QSO where no
     line is detected with ALMA.  The black stars correspond to the
     [Fujimoto, Brammer and Watson (2022)]() and [Endsley, Stark and Lyu (2023)]() QSO that
     were identified in deep multiwavelength data over legacy fields
     and not from wide-area QSO searches.  Shown along the right
     vertical axis is the approximate gas mass one would expect for
     the host galaxy of a given [CII] luminosity using the canonical
     [Zanella, Daddi and Magdis (2018)]() relation.  $\sim$21\% of $UV$-faint QSOs
     show luminous [CII] emission and thus appear to reside in massive
     host galaxies.  (_lower panel_) Number of [CII]
     observations for $z>5.9$ QSOs vs. $UV$ luminosity from the
     literature (_filled black histogram_) and from CISTERN
     (_filled red histogram_).  Thanks to observations from
     CISTERN, there has been a dramatic $\sim$5$\times$ increase in
     coverage of [CII] in $UV$-faint ($M_{UV,AB}>-24.5$) QSOs with ALMA.
    (*fig:muvcii*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.24128"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

547  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
